In [12]:
#data
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit

#aux
import os
import pickle

# essas duas linhas são usadas para definir como os números de ponto flutuante 
# serão formatados ao serem exibidos em numpy e pandas, garantindo que eles sejam 
# exibidos com uma precisão específica e sem notação científica.'''
np.set_printoptions(suppress=True, formatter={'float_kind':'{:16.3f}'.format}, linewidth=130)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

## carregar os dados

In [49]:
# load the dataframe object from the pickle file
with open(os.path.join('data', 'df_itens_geral_validos.pkl'), 'rb') as f:
    dataset = pickle.load(f)

## selecionar as colunas de interesse

In [50]:
dataset = dataset.loc[:, ['tf_idf', 'wes', 'wes_own', 'classes_param_a_1', 'classes_param_b_1', 'classes_param_c_1']]

## trocar os valores das classes pelos valores numéricos

In [51]:
dataset['classes_param_a_1'].unique()

['>2.092 e <=2.735', '<=1.581', '>1.581 e <=2.092', '>2.735']
Categories (4, object): ['<=1.581' < '>1.581 e <=2.092' < '>2.092 e <=2.735' < '>2.735']

In [52]:
# Criar dicionário de mapeamento
mapping = {'<=1.581': 0, '>1.581 e <=2.092': 1, '>2.092 e <=2.735': 2, '>2.735': 3}

# Substituir valores na coluna
dataset['classes_param_a_1'] = dataset['classes_param_a_1'].replace(mapping)

In [53]:
dataset['classes_param_b_1'].unique()

['>0.652 e <=1.224', '>1.224 e <=1.837', '<=0.652', '>1.837']
Categories (4, object): ['<=0.652' < '>0.652 e <=1.224' < '>1.224 e <=1.837' < '>1.837']

In [54]:
# Criar dicionário de mapeamento
mapping = {'<=0.652': 0, '>0.652 e <=1.224': 1, '>1.224 e <=1.837': 2, '>1.837': 3}

# Substituir valores na coluna
dataset['classes_param_b_1'] = dataset['classes_param_b_1'].replace(mapping)

In [55]:
dataset['classes_param_c_1'].unique()

['>0.203', '>0.165 e <=0.203', '<=0.12487', '>0.12487 e <=0.165']
Categories (4, object): ['<=0.12487' < '>0.12487 e <=0.165' < '>0.165 e <=0.203' < '>0.203']

In [56]:
# Criar dicionário de mapeamento
mapping = {'<=0.12487': 0, '>0.12487 e <=0.165': 1, '>0.165 e <=0.203': 2, '>0.203': 3}

# Substituir valores na coluna
dataset['classes_param_c_1'] = dataset['classes_param_c_1'].replace(mapping)

## dividir os dados entre treino e teste

A classe **StratifiedShuffleSplit** é usada para criar uma amostra estratificada de 20% dos dados para teste e 80% dos dados para treinamento, com base nas proporções de valores de uma coluna.

É uma técnica de validação cruzada que embaralha os dados de maneira estratificada.

- n_splits=1: especifica o número de repetições da validação cruzada. Neste caso, a validação cruzada será realizada apenas uma vez.
- test_size=0.2: indica a proporção de amostras que devem ser incluídas no conjunto de testes. Neste caso, 20% das amostras serão usadas para teste.
- random_state=0: é um número que define a semente do gerador de números aleatórios para garantir que os resultados possam ser reproduzidos.

Cria um objeto que pode ser usado para dividir um conjunto de dados em um conjunto de treinamento e um conjunto de teste de maneira estratificada e com uma proporção de amostras de teste especificada.

In [57]:
dataset.head()

,tf_idf,wes,wes_own,classes_param_a_1,classes_param_b_1,classes_param_c_1
0,"[0.15448001662671257, 0.19314334362072388, 0.1...","[[0.0087, -0.015, 0.073, 0.0244, -0.0139, -0.0...","[[-0.2735036, -0.19421376, 0.26711008, 0.26224...",2,1,3
1,"[0.14463912531808382, 0.1882090195082598, 0.11...","[[-0.0457, -0.009, 0.0753, -0.0198, -0.0479, -...","[[0.33006182, 0.08265074, 0.0863686, 0.1423417...",0,2,3
2,"[0.08937166525923214, 0.12822456358498585, 0.1...","[[-0.0246, -0.0379, 0.1021, 0.0378, -0.0864, -...","[[0.2161539, 0.16423477, -0.022779496, 0.32457...",1,2,2
3,"[0.07309805460729854, 0.07309805460729854, 0.0...","[[-0.0016, -0.0689, 0.0547, -0.0424, 0.0693, -...","[[-0.06490169, 0.25573078, 0.082524896, 0.1681...",1,0,2
4,"[0.238915125308662, 0.2103826054875806, 0.2674...","[[-0.0128, -0.0053, 0.029, -0.052, -0.0304, -0...","[[-0.123734914, 0.036533087, 0.20261528, 0.195...",0,2,0


In [58]:
# função para estratificar os dados criando os conjuntos de treino e teste
def estratificacao_train_test_por_classe(df, col_class, nome_dataset):
    y = df[col_class]
    #print(y.unique())
    # Criando o objeto StratifiedShuffleSplit
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=0)

    # Iterando pelos splits e separando a amostra
    for train_index, test_index in splitter.split(df, y):
        df_train = df.iloc[train_index]
        df_test = df.iloc[test_index]
        
    file = os.path.join('data', nome_dataset + '_train.pkl')
    # save the dataframe object to a pickle file
    with open(file, 'wb') as f:
        pickle.dump(df_train, f)
    
    file = os.path.join('data', nome_dataset + '_test.pkl')
    # save the dataframe object to a pickle file
    with open(file, 'wb') as f:
        pickle.dump(df_test, f)
    
    return df_train, df_test

In [59]:
estratificacao_train_test_por_classe(dataset, 'classes_param_a_1', 'dataset_param_a')

(                                                 tf_idf  \
 1760  [0.12456968494564394, 0.08988356079585649, 0.1...   
 564   [0.05662525107280963, 0.10048547503414434, 0.1...   
 401   [0.026692991743747074, 0.05267474243888511, 0....   
 1876  [0.046029406346867315, 0.060844034470711586, 0...   
 956   [0.08829657106261818, 0.060500120867303185, 0....   
 ...                                                 ...   
 456   [0.4304220081037565, 0.22943255222025502, 0.20...   
 1515  [0.09869963709433056, 0.06062230642955486, 0.0...   
 216   [0.07839304093995585, 0.05048618299470653, 0.0...   
 1600  [0.10465107118200082, 0.11563591269664297, 0.0...   
 1417  [0.07533008177921344, 0.06787074975830051, 0.0...   
 
                                                     wes  \
 1760  [[-0.0073, -0.0428, -0.0087, 0.035, 0.0179, -0...   
 564   [[-0.0225, -0.0023, 0.0529, 0.0049, 0.042, -0....   
 401   [[-0.084, 0.0216, 0.0036, -0.034, -0.0106, -0....   
 1876  [[0.03, 0.0158, 0.0584, 0.0091,

In [60]:
estratificacao_train_test_por_classe(dataset, 'classes_param_b_1', 'data_set_param_b')

(                                                 tf_idf  \
 1757  [0.1520375028210196, 0.17699962659034899, 0.11...   
 573   [0.17117729139134427, 0.12149402947247992, 0.1...   
 357   [0.056354022158337326, 0.0900773086241686, 0.0...   
 1874  [0.06280307748585236, 0.5022780351753088, 0.05...   
 952   [0.2162069029386577, 0.17003122409083568, 0.21...   
 ...                                                 ...   
 469   [0.03344511075801128, 0.03344511075801128, 0.0...   
 1523  [0.1444444525285251, 0.14320885524210786, 0.11...   
 184   [0.07487472267235452, 0.08646331108472735, 0.0...   
 1623  [0.24468841743329098, 0.07221445715635107, 0.0...   
 1442  [0.11702273888350695, 0.09765272329520391, 0.0...   
 
                                                     wes  \
 1757  [[-0.0024, -0.033, 0.0099, 0.0034, -0.1078, -0...   
 573   [[-0.0312, -0.0228, 0.0538, 0.2069, -0.0707, -...   
 357   [[0.0087, -0.015, 0.073, 0.0244, -0.0139, -0.0...   
 1874  [[0.0033, 0.0033, 0.0621, -0.01

In [61]:
estratificacao_train_test_por_classe(dataset, 'classes_param_c_1', 'dataset_param_c')

(                                                 tf_idf  \
 1726  [0.10125090615341521, 0.10326455307979217, 0.0...   
 577   [0.11111185982446523, 0.06057437140247389, 0.0...   
 469   [0.03344511075801128, 0.03344511075801128, 0.0...   
 1898  [0.0689360865233989, 0.06312408820152467, 0.05...   
 1020  [0.1971223244537432, 0.09724864647043395, 0.18...   
 ...                                                 ...   
 554   [0.08816266077549254, 0.09613017276934961, 0.0...   
 927   [0.10552815671310119, 0.11907743640086556, 0.0...   
 250   [0.07240124363500232, 0.06347045699661816, 0.0...   
 1598  [0.07874032035267263, 0.05730314456326236, 0.4...   
 1465  [0.3352598583222355, 0.09739169217820315, 0.07...   
 
                                                     wes  \
 1726  [[0.0028, -0.0298, 0.0048, 0.002, 0.0115, -0.0...   
 577   [[-0.1424, 0.0612, 0.0241, -0.1269, -0.0183, 0...   
 469   [[-0.022, 0.0315, 0.0752, -0.0268, 0.0625, -0....   
 1898  [[0.0771, -0.0297, -0.0256, 0.0